In [10]:
import pandas as pd
import requests

In [20]:
# Define the URL and headers
url = "https://eu-offering-api.kambicdn.com/offering/v2018/ubnl/listView/football/england/premier_league/all/matches.json"
params = {
    "lang": "nl_NL",
    "market": "NL",
    "client_id": "2",
    "channel_id": "1",
    "ncid": "1730835538397",
    "useCombined": "true"
}

headers = {
    "Accept": "application/json, text/javascript, */*; q=0.01",
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Accept-Language": "en-US,en;q=0.9",
    "Origin": "https://www.unibet.nl",
    "Referer": "https://www.unibet.nl/",
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36"
}

# Make the GET request
response = requests.get(url, headers=headers, params=params)

# Check the status code
if response.status_code == 200:
    # Parse the JSON response
    eventsData = response.json()
    print(eventsData)  # Output the data for inspection
else:
    print(f"Request failed with status code: {response.status_code}")

{'events': [{'event': {'id': 1021042522, 'name': 'West Ham - Everton', 'nameDelimiter': '-', 'englishName': 'West Ham - Everton', 'homeName': 'West Ham', 'awayName': 'Everton', 'start': '2024-11-09T15:00:00Z', 'group': 'Premier League', 'groupId': 1000094985, 'path': [{'id': 1000093190, 'name': 'Voetbal', 'englishName': 'Football', 'termKey': 'football'}, {'id': 1000461733, 'name': 'Engeland', 'englishName': 'England', 'termKey': 'england'}, {'id': 1000094985, 'name': 'Premier League', 'englishName': 'Premier League', 'termKey': 'premier_league'}], 'nonLiveBoCount': 170, 'sport': 'FOOTBALL', 'tags': ['OFFERED_LIVE', 'BET_BUILDER', 'MATCH'], 'state': 'NOT_STARTED'}, 'betOffers': [{'id': 2468614713, 'closed': '2024-11-09T15:00:00Z', 'criterion': {'id': 1001159858, 'label': 'Reguliere Speeltijd', 'englishLabel': 'Full Time', 'order': [], 'occurrenceType': 'GOALS', 'lifetime': 'FULL_TIME'}, 'betOfferType': {'id': 2, 'name': 'Wedstrijd ', 'englishName': 'Match'}, 'eventId': 1021042522, 'out

In [24]:
# Extract the relevant data
rows = []

for event in eventsData['events']:
    event_info = event['event']
    for offer in event['betOffers']:
        for outcome in offer['outcomes']:
            row = {
                'event_id': event_info['id'],
                'event_name': event_info['name'],
                'home_team': event_info['homeName'],
                'away_team': event_info['awayName'],
                'start_time': event_info['start'],
                'bet_offer_id': offer['id'],
                'bet_offer_type_name': offer['betOfferType']['name'],
                'criterion_label': offer['criterion']['label'],
                'outcome_id': outcome['id'],
                'outcome_label': outcome['label'],
                'participant': outcome.get('participant', None),
                'odds': outcome['odds'],
                'odds_fractional': outcome['oddsFractional'],
                'odds_american': outcome['oddsAmerican'],
                'status': outcome['status'],
                'cash_out_status': outcome['cashOutStatus']
            }
            rows.append(row)

# Create the DataFrame
events_df = pd.DataFrame(rows)

In [26]:
# Display the DataFrame
events_df.to_csv('unibet_events_df.csv')

## Loop through event IDs

In [29]:
import requests
import pandas as pd

# Assuming `events_df` is the DataFrame you've created from the previous example

# Get unique event IDs from the DataFrame
unique_event_ids = events_df['event_id'].unique()

# Define the base URL and parameters
base_url = 'https://eu-offering-api.kambicdn.com/offering/v2018/ubnl/betoffer/event/{event_id}.json'
params = {
    'lang': 'nl_NL',
    'market': 'NL',
    'client_id': '2',
    'channel_id': '1',
    'ncid': '1730829230569',
    'includeParticipants': 'true'
}
headers = {
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'accept-encoding': 'gzip, deflate, br, zstd',
    'accept-language': 'en-US,en;q=0.9',
    'origin': 'https://www.unibet.nl',
    'priority': 'u=1, i',
    'referer': 'https://www.unibet.nl/',
    'sec-ch-ua': '"Google Chrome";v="129", "Not=A?Brand";v="8", "Chromium";v="129"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'cross-site',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36'
}

# Initialize an empty list to collect all rows
all_rows = []

# Loop through each unique event_id and make the request
for event_id in unique_event_ids:
    # Construct the URL with the current event_id
    url = base_url.format(event_id=event_id)
    
    # Make the request
    response = requests.get(url, headers=headers, params=params)

    # Check the response
    if response.status_code == 200:
        data = response.json()  # Parse the JSON response
        bet_offers = data.get('betOffers', [])  # Safely get the betOffers list
        
        for offer in bet_offers:
            for outcome in offer['outcomes']:
                row = {
                    'bet_offer_id': offer['id'],
                    'closed': offer['closed'],
                    'criterion_id': offer['criterion']['id'],
                    'criterion_label': offer['criterion']['label'],
                    'criterion_english_label': offer['criterion'].get('englishLabel', None),  # Safe access
                    'occurrence_type': offer['criterion'].get('occurrenceType', None),  # Safe access
                    'lifetime': offer['criterion'].get('lifetime', None),  # Safe access
                    'bet_offer_type_id': offer['betOfferType']['id'],
                    'bet_offer_type_name': offer['betOfferType']['name'],
                    'bet_offer_type_english_name': offer['betOfferType']['englishName'],
                    'event_id': offer['eventId'],
                    'outcome_id': outcome['id'],
                    'outcome_label': outcome['label'],
                    'outcome_english_label': outcome['englishLabel'],
                    'odds': outcome.get('odds', None),  # Safe access
                    'line': outcome.get('line', None),  # New field added
                    'participant': outcome.get('participant', None),  # New field added
                    'type': outcome['type'],
                    'changed_date': outcome['changedDate'],
                    'odds_fractional': outcome.get('oddsFractional', None),  # Safe access
                    'odds_american': outcome.get('oddsAmerican', None),  # Safe access
                    'status': outcome['status'],
                    'cash_out_status': outcome['cashOutStatus'],
                    'home_score': outcome.get('homeScore', None),  # Safe access
                    'away_score': outcome.get('awayScore', None)   # Safe access
                }
                all_rows.append(row)  # Append the row to the list

    else:
        print(f"Request for event_id {event_id} failed with status code:", response.status_code)

# After the loop, create the final DataFrame from all collected rows
final_df = pd.DataFrame(all_rows)

      bet_offer_id                closed  criterion_id  \
0       2468595399  2024-11-09T15:00:00Z    1000505272   
1       2468595399  2024-11-09T15:00:00Z    1000505272   
2       2468595399  2024-11-09T15:00:00Z    1000505272   
3       2468595399  2024-11-09T15:00:00Z    1000505272   
4       2468595399  2024-11-09T15:00:00Z    1000505272   
...            ...                   ...           ...   
7400    2470304587  2024-11-25T20:00:00Z    1001159668   
7401    2470304587  2024-11-25T20:00:00Z    1001159668   
7402    2470304587  2024-11-25T20:00:00Z    1001159668   
7403    2470304588  2024-11-25T20:00:00Z    1001159884   
7404    2470304588  2024-11-25T20:00:00Z    1001159884   

                criterion_label   criterion_english_label occurrence_type  \
0     Correcte Score - 1e Helft  Correct Score - 1st Half           GOALS   
1     Correcte Score - 1e Helft  Correct Score - 1st Half           GOALS   
2     Correcte Score - 1e Helft  Correct Score - 1st Half           GOAL

In [30]:
final_df

,bet_offer_id,closed,criterion_id,criterion_label,criterion_english_label,occurrence_type,lifetime,bet_offer_type_id,bet_offer_type_name,bet_offer_type_english_name,...,line,participant,type,changed_date,odds_fractional,odds_american,status,cash_out_status,home_score,away_score
0,2468595399,2024-11-09T15:00:00Z,1000505272,Correcte Score - 1e Helft,Correct Score - 1st Half,GOALS,None,3,Correcte Score,Correct Score,...,NaN,None,OT_UNTYPED,2024-11-05T14:10:33Z,43/20,215,OPEN,ENABLED,0,0
1,2468595399,2024-11-09T15:00:00Z,1000505272,Correcte Score - 1e Helft,Correct Score - 1st Half,GOALS,None,3,Correcte Score,Correct Score,...,NaN,None,OT_UNTYPED,2024-11-05T14:10:33Z,3/1,300,OPEN,ENABLED,1,0
2,2468595399,2024-11-09T15:00:00Z,1000505272,Correcte Score - 1e Helft,Correct Score - 1st Half,GOALS,None,3,Correcte Score,Correct Score,...,NaN,None,OT_UNTYPED,2024-11-05T14:10:33Z,17/2,850,OPEN,ENABLED,2,0
3,2468595399,2024-11-09T15:00:00Z,1000505272,Correcte Score - 1e Helft,Correct Score - 1st Half,GOALS,None,3,Correcte Score,Correct Score,...,NaN,None,OT_UNTYPED,2024-11-05T14:10:33Z,30/1,3000,OPEN,ENABLED,3,0
4,2468595399,2024-11-09T15:00:00Z,1000505272,Correcte Score - 1e Helft,Correct Score - 1st Half,GOALS,None,3,Correcte Score,Correct Score,...,NaN,None,OT_UNTYPED,2024-11-05T14:10:33Z,160/1,16000,OPEN,ENABLED,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7400,2470304587,2024-11-25T20:00:00Z,1001159668,Dubbele Kans - 1e Helft,Double Chance - 1st Half,GOALS,None,12,Dubbele kans,Double Chance,...,NaN,None,OT_ONE_OR_CROSS,2024-11-04T08:33:59Z,2/15,-715,OPEN,ENABLED,None,None
7401,2470304587,2024-11-25T20:00:00Z,1001159668,Dubbele Kans - 1e Helft,Double Chance - 1st Half,GOALS,None,12,Dubbele kans,Double Chance,...,NaN,None,OT_ONE_OR_TWO,2024-11-04T08:33:59Z,4/9,-225,OPEN,ENABLED,None,None
7402,2470304587,2024-11-25T20:00:00Z,1001159668,Dubbele Kans - 1e Helft,Double Chance - 1st Half,GOALS,None,12,Dubbele kans,Double Chance,...,NaN,None,OT_CROSS_OR_TWO,2024-11-04T08:33:59Z,13/20,-152,OPEN,ENABLED,None,None
7403,2470304588,2024-11-25T20:00:00Z,1001159884,Draw No Bet - 1e Helft,Draw No Bet - 1st Half,GOALS,None,2,Wedstrijd,Match,...,NaN,Newcastle United,OT_ONE,2024-11-04T08:33:59Z,3/10,-335,OPEN,ENABLED,None,None
